<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5_1300ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jan 16 08:13:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install simpletransformers
!pip install torch
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd

data = pd.read_csv('/content/gdrive/MyDrive/sentences_data.csv')
data

,English,Thai
0,There are several agricultural residues in the...,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...
1,"At present, biomass is being used in the small...",ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก
2,It is an effective way to eliminate agricultur...,เป็นวิธีที่มีประสิทธิภาพในการกำจัดของเสียทางกา...
3,"Recently, there are many incentives from the g...",เมื่อเร็ว ๆ นี้มีแรงจูงใจมากมายจากรัฐบาลสำหรับ...
4,"For examples, these electricity producers are ...",ตัวอย่างเช่น ผู้ผลิตไฟฟ้าเหล่านี้ได้รับอนุญาตใ...
...,...,...
1288,But the laws require the energy management wor...,แต่กฎหมายกำหนดให้คณะทำงานด้านการจัดการพลังงานต...
1289,The objective of the research is to develop me...,วัตถุประสงค์ของงานวิจัยคือการพัฒนาวิธีการในการ...
1290,"In this work, the control charts for individua...",ในงานนี้แผนภูมิควบคุมสำหรับข้อมูลเชิงเดียวและค...
1291,The criterion used in determining the achievem...,เกณฑ์ที่ใช้ในการพิจารณาผลสัมฤทธิ์จากมาตรการฯ ค...


In [5]:
import csv

with open('/content/dataset.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['prefix', 'input_text', 'target_text'])
  for i in range(len(data)):
    writer.writerow(['translate thai to english', data['Thai'][i], data['English'][i]])
    writer.writerow(['translate english to thai', data['English'][i], data['Thai'][i]])


pd.read_csv('/content/dataset.csv')

,prefix,input_text,target_text
0,translate thai to english,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...,There are several agricultural residues in the...
1,translate english to thai,There are several agricultural residues in the...,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...
2,translate thai to english,ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก,"At present, biomass is being used in the small..."
3,translate english to thai,"At present, biomass is being used in the small...",ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก
4,translate thai to english,เป็นวิธีที่มีประสิทธิภาพในการกำจัดของเสียทางกา...,It is an effective way to eliminate agricultur...
...,...,...,...
2581,translate english to thai,"In this work, the control charts for individua...",ในงานนี้แผนภูมิควบคุมสำหรับข้อมูลเชิงเดียวและค...
2582,translate thai to english,เกณฑ์ที่ใช้ในการพิจารณาผลสัมฤทธิ์จากมาตรการฯ ค...,The criterion used in determining the achievem...
2583,translate english to thai,The criterion used in determining the achievem...,เกณฑ์ที่ใช้ในการพิจารณาผลสัมฤทธิ์จากมาตรการฯ ค...
2584,translate thai to english,ผลการวิเคราะห์แผนภูมิควบคุมพบว่ามาตรการที่ส่งผ...,The analyzed result of control charts showed t...


In [6]:
!rm -rf '/content/train.csv'
!rm -rf '/content/eval.csv'
!rm -rf '/content/test.csv'

from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/dataset.csv')
train_data, eval_test = train_test_split(data, train_size=0.9, shuffle=False)
train_data.to_csv('/content/train.csv', index=False)
eval_data, test_data = train_test_split(eval_test, train_size=0.9, shuffle=False)
eval_data.to_csv('/content/eval.csv', index=False)
test_data.to_csv('/content/test.csv', index=False)

In [7]:
pd.read_csv('/content/train.csv')

,prefix,input_text,target_text
0,translate thai to english,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...,There are several agricultural residues in the...
1,translate english to thai,There are several agricultural residues in the...,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...
2,translate thai to english,ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก,"At present, biomass is being used in the small..."
3,translate english to thai,"At present, biomass is being used in the small...",ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก
4,translate thai to english,เป็นวิธีที่มีประสิทธิภาพในการกำจัดของเสียทางกา...,It is an effective way to eliminate agricultur...
...,...,...,...
2322,translate thai to english,การประเมินประสิทธิภาพใช้ผู้เชี่ยวชาญ 5 คน และก...,The performance appraisal was evaluated by 5 e...
2323,translate english to thai,The performance appraisal was evaluated by 5 e...,การประเมินประสิทธิภาพใช้ผู้เชี่ยวชาญ 5 คน และก...
2324,translate thai to english,เครื่องมือที่ใช้เป็นแบบประเมินประสิทธิภาพและแบ...,The tools used this research were performance ...
2325,translate english to thai,The tools used this research were performance ...,เครื่องมือที่ใช้เป็นแบบประเมินประสิทธิภาพและแบ...


In [8]:
pd.read_csv('/content/eval.csv')

,prefix,input_text,target_text
0,translate english to thai,The data were analyzed by mean and standard de...,สถิติที่ใช้ในการวิเคราะห์ข้อมูล ได้แก่ ค่าเฉลี...
1,translate thai to english,งานวิจัยนี้เป็นการศึกษาความหลากหลายของชนิดเห็ด...,This survey research was aimed to study the di...
2,translate english to thai,This survey research was aimed to study the di...,งานวิจัยนี้เป็นการศึกษาความหลากหลายของชนิดเห็ด...
3,translate thai to english,เป็นการศึกษาเชิงสำรวจ โดยทำการศึกษารูปพรรณสัณฐ...,It was also to study the morphological charact...
4,translate english to thai,It was also to study the morphological charact...,เป็นการศึกษาเชิงสำรวจ โดยทำการศึกษารูปพรรณสัณฐ...
...,...,...,...
228,translate english to thai,The results of the literature review and analy...,ซึ่งผลจากการทบทวนวรรณกรรมและการวิเคราะห์ พบว่า...
229,translate thai to english,1) หน่วยงานด้านการศึกษา 2) หน่วยงานด้านการท่อง...,1) Education agency 2) tourism agency 3) publi...
230,translate english to thai,1) Education agency 2) tourism agency 3) publi...,1) หน่วยงานด้านการศึกษา 2) หน่วยงานด้านการท่อง...
231,translate thai to english,ซึ่งแนวทางเพื่อเพิ่มประสิทธิภาพในองค์กรด้วยกา...,The approach to increase efficiency in the or...


In [9]:
pd.read_csv('/content/test.csv')

,prefix,input_text,target_text
0,translate thai to english,1) การลดระยะเวลา 2) การลดความผิดพลาด 3) การลดต...,1) Reducing time 2) Reducing errors 3) Reducin...
1,translate english to thai,1) Reducing time 2) Reducing errors 3) Reducin...,1) การลดระยะเวลา 2) การลดความผิดพลาด 3) การลดต...
2,translate thai to english,งานวิจัยนี้มีวัตถุประสงค์เพื่อศึกษาแบบจำลองการ...,The purpose of this research is to study the w...
3,translate english to thai,The purpose of this research is to study the w...,งานวิจัยนี้มีวัตถุประสงค์เพื่อศึกษาแบบจำลองการ...
4,translate thai to english,โดยพิจารณาสัดส่วนของกลไกการก้าวเดินตามหลักการ...,The proportions of the walking mechanism are c...
5,translate english to thai,The proportions of the walking mechanism are c...,โดยพิจารณาสัดส่วนของกลไกการก้าวเดินตามหลักการ...
6,translate thai to english,อาศัยหลักการการเคลื่อนที่ของกลไกแขนเชื่อมโยง 4...,The principle of movement of the four – bar li...
7,translate english to thai,The principle of movement of the four – bar li...,อาศัยหลักการการเคลื่อนที่ของกลไกแขนเชื่อมโยง 4...
8,translate thai to english,แบบจำลองการก้าวเดินที่ได้ถูกนำไปใช้ในการสร้างต...,"The walking model, which has been used to buil..."
9,translate english to thai,"The walking model, which has been used to buil...",แบบจำลองการก้าวเดินที่ได้ถูกนำไปใช้ในการสร้างต...


In [10]:
import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.read_csv('/content/train.csv').astype(str)
eval_df = pd.read_csv('/content/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 8
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Thai-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)

In [11]:
# !wandb login --relogin

In [12]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/2327 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: defymiygfl (sigmarai). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 8:   0%|          | 0/291 [00:00<?, ?it/s]

...

Running Epoch 6 of 8:   0%|          | 0/291 [00:00<?, ?it/s]

  0%|          | 0/233 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/291 [00:00<?, ?it/s]

  0%|          | 0/233 [00:00<?, ?it/s]

(2328,
 {'global_step': [291, 582, 873, 1164, 1455, 1746, 2037, 2328],
  'eval_loss': [3.5109379053115846,
   3.22568359375,
   3.282303063074748,
   3.3577263275782268,
   3.5391404072443646,
   3.7681894143422445,
   4.026313360532125,
   4.17799535592397],
  'train_loss': [3.892975091934204,
   2.9605066776275635,
   2.615412473678589,
   2.008971691131592,
   1.760765552520752,
   1.0537222623825073,
   0.7707270979881287,
   1.0596308708190918]})

In [13]:
# Evaluate the model
result = model.eval_model(eval_df)

  0%|          | 0/233 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

In [14]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 96
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs", args=model_args)

In [15]:
valid = pd.read_csv("/content/eval.csv").astype(str)

thai_truth = [valid.loc[valid["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = valid.loc[valid["prefix"] == "translate english to thai"]["input_text"].tolist()

english_truth = [valid.loc[valid["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = valid.loc[valid["prefix"] == "translate thai to english"]["input_text"].tolist()

In [16]:
english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)

thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

Generating outputs:   0%|          | 0/15 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/116 [00:00<?, ?it/s]

Thai to English:  10.599384375775438


Generating outputs:   0%|          | 0/15 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/117 [00:00<?, ?it/s]

English to Thai:  1.7125308554485517


In [17]:
from simpletransformers.t5 import T5Model

# model = T5Model("mt5", "/content/outputs")
to_predict = ['การสร้างเครื่องแปลภาษาเป็นโจทย์ classic ของ NLP และสามารถนำไปใช้ประโยชน์ได้โดยตรงโดยตัวของมันเอง โจทย์นี้จัดว่าเป็นโจทย์ที่ยากที่สุดของ NLP เพราะต้องอาศัยความเข้าใจของภาษาในระดับที่ลึกซึ้งทำให้โมเดลที่ต้องใช้ต้องมีความซับซ้อนตามไปด้วย']
to_predict = to_predict[0].split()
predictions = model.predict(to_predict)
predictions

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/8 [00:00<?, ?it/s]

['The creation of the voice speaking machine is considered.',
 'The objectives of this research were',
 'The objectives of this research were',
 'NLP.',
 'And can be used to benefit directly with the equation.',
 'This is considered to be the most common suitable suitable s.',
 'NLP.',
 'Because it is necessary to know the logical speaking in the depth of the locus of complexity.']

In [ ]:
to_predict[0].split()

In [ ]:
# test_data['input_text']

In [ ]:
# with open('/content/predict.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['ref_th', 'ref_eng', 'predict'])
#   for i in range(len(test_data)):
#     writer.writerow([test_data['input_text'][i], test_data['target_text'][i], predictions[i]])
# pd.read_csv('/content/predict.csv')